In [ ]:
# myfile_cameras = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/cameras.bin'
# myfile_cameras = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/cameras.bin'
# myfile_cameras = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/cameras.bin'

# import numpy as np

# dtype = np.dtype('B')
# try:
#     with open(myfile_cameras, "rb") as f:
#         numpy_data = np.fromfile(f,dtype)
#     print(numpy_data)
# except IOError:
#     print('Error While Opening the file!')    

# print(numpy_data.shape)
# for i in range(numpy_data.shape[0]):
#     print(i, numpy_data[i])

In [1]:
import numpy as np
import open3d as o3d
import sympy as sp
import cv2
import csv
import sys

myfile_cameras = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/cameras.txt'
myfile_images = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/images.txt'
myfile_points3D = '/home/shivani/Documents/BuildingInspection/SFM/sparse/0/points3D.txt'

'''
cameras:
CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[single focal length, principal point x, principlal point y, distortion coeff]

images: (Image list with two lines of data per image:)
IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
POINTS2D[] as (X, Y, POINT3D_ID)

key 2d points used in the image are in POINTS2D and each of these points are related to a 3d point
if POINT3D_ID = -1, it means that 2d point is not used in reconstruction

points3D: (3D point list with one line of data per point:)
POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)

The error is given in pixels of reprojection error 
and is only updated after global bundle adjustment

'''

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'\ncameras:\nCAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[single focal length, principal point x, principlal point y, distortion coeff]\n\nimages: (Image list with two lines of data per image:)\nIMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME\nPOINTS2D[] as (X, Y, POINT3D_ID)\n\nkey 2d points used in the image are in POINTS2D and each of these points are related to a 3d point\nif POINT3D_ID = -1, it means that 2d point is not used in reconstruction\n\npoints3D: (3D point list with one line of data per point:)\nPOINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)\n\nThe error is given in pixels of reprojection error \nand is only updated after global bundle adjustment\n\n'

In [2]:
def quaternion_rotation_matrix(Q, T):
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
    
    tx = T[0]
    ty = T[1]
    tz = T[2]
    
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
    
    translation_matrix = np.array([[tx],
                                   [ty],
                                   [tz]])
                            
    transformation_matrix = getTransformationMatrix(rot_matrix, translation_matrix)
    return rot_matrix, translation_matrix, transformation_matrix

def getTransformationMatrix(R, t):
    # given Rotation matrix, R and translation matrix, t
    # return the transformation matrix describing the combined motion, T
    augment = np.array([0, 0, 0, 1]) # Converting to homogeneous matrix
    R = np.array(R)
    t = np.array(t)
    if (t.shape == (3,)):
        t = t.reshape((3,1))
    T = np.hstack((R, t))
    T = np.vstack((T, augment))
    assert(T.shape, (4,4))
    return T

def getCameraIntrinsicMatrix(params):
    # focal length, principlal point x, principal point y, distortion param 
    K = np.identity(3) # K: Callibration Matrix
    K[0, 0] = params[0]
    K[1, 1] = params[1]
    K[0, 2] = params[2]
    K[1, 2] = params[3]
    return K


<>:48: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:48: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:48: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-2-89b5428bbe43>:48: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(T.shape, (4,4))


In [3]:
imname = '00030.jpg'
imfile = '/home/shivani/Documents/BuildingInspection/' + imname
img = cv2.imread(imfile)
cv2.imshow('image',img)

coordinates = []
def click_event(event, x, y, flags, params):
  
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x, ' ', y)  
        cv2.imshow('image', img)
        coordinates.append([x,y])
  
    # checking for right mouse clicks     
    if event==cv2.EVENT_RBUTTONDOWN:
        print(x, ' ', y)
        cv2.imshow('image', img)
        coordinates.append([x,y])

# setting mouse hadler for the image
# and calling the click_event() function
cv2.setMouseCallback('image', click_event)

# wait for a key to be pressed to exit
cv2.waitKey(0)

# close the window
cv2.destroyAllWindows()

628   424
658   260
884   180
1073   525
1288   417
715   83
410   214


In [4]:
print(len(coordinates))

7


In [5]:
###### parse sfm files -- points3D.txt, cameras.txt and images.txt
csv.field_size_limit(sys.maxsize)

with open(myfile_points3D, 'r') as fd:
    reader = csv.reader(fd)
    rowArr = []
    points_3d_ids = []
    xyz_coords = []
    rgb_vals = []
    reproj_errs = []
    tracks = []
    for i,line in enumerate(reader):
        rowArr.append(list(map(float, line[0].split())))
        points_3d_ids.append(line[0].split()[0])
        xyz_coords.append(line[0].split()[1:4])
        rgb_vals.append(line[0].split()[4:7])
        reproj_errs.append(line[0].split()[7])
        tracks.append(line[0].split()[8:]) #(IMAGE_ID, POINT2D_IDX)
        
twoDPoints = rowArr[1]
print('No. of 2DPoints: ', len(twoDPoints))
print(twoDPoints)
print(len(points_3d_ids))

No. of 2DPoints:  20
[370410.0, 0.237232, -0.440943, 5.26491, 121.0, 122.0, 117.0, 3.28102, 160.0, 1610.0, 161.0, 907.0, 155.0, 3552.0, 152.0, 3298.0, 159.0, 4882.0, 157.0, 6437.0]
263453


In [6]:
csv.field_size_limit(sys.maxsize)

with open(myfile_cameras, 'r') as fd2:
    reader2 = csv.reader(fd2)
    rowArr2 = []
    camera_ids = []
    width = 0
    height = 0
    params = [] # f, cx, cy, dist
    for i,line in enumerate(reader2):
#         rowArr2.append(list(map(float, line[0].split())))
        rowArr2.append(list(line[0].split()))
        camera_ids.append(line[0].split()[0])
        params.append(line[0].split()[4:]) #(IMAGE_ID, POINT2D_IDX)
        if (i == 0):
            width = line[0].split()[2]
            height = line[0].split()[3]
            
print(len(camera_ids))
print(rowArr2[1])

400
['399', 'SIMPLE_RADIAL', '1920', '1080', '1534.87', '960', '540', '0.00945933']


In [7]:
csv.field_size_limit(sys.maxsize)

with open(myfile_images, 'r') as fd3:
    reader3 = csv.reader(fd3)
    rowArr3 = []
    rowArr3_2 = []
    image_ids = []
    Quats = []
    Ts = []
    camera_ids_of_images = []
    image_names = []
    
#     keypoint_coords_all = [] # with 3d point id
#     keypoint_coords_used = [] # with 3d point id
    keypoints_x = []
    keypoints_y = []
    inds_3d_point = []
    
    for i,line in enumerate(reader3):
#         rowArr3.append(list(map(float, line[0].split())))
#         print('i',i)
        if (i == 0):
            width = line[0].split()[2]
            height = line[0].split()[3]
        if(i%2 == 0): # lines 0,2,4 ...
            rowArr3.append(list(line[0].split()))
            image_ids.append(line[0].split()[0])
            Quats.append(line[0].split()[1:5])
            Ts.append(line[0].split()[5:8])
            camera_ids_of_images.append(line[0].split()[8])
            image_names.append(line[0].split()[9])
        elif(i%2 == 1):
            rowArr3_2.append(list(line[0].split()))
            keypoints_x.append(line[0].split()[0::3])
            keypoints_y.append(line[0].split()[1::3])
            inds_3d_point.append(line[0].split()[2::3])
# print(len(camera_ids_of_images))
# print(rowArr3[1])
# print(rowArr3[2])

In [ ]:
## filter un-used keypoints ie remove 2d points with 3d point index -1

inds_used = []
keypoints_x_used = []
keypoints_y_used = []

# inds_3d_point = list(map(int,inds_3d_point))
# keypoints_x = list(map(float,keypoints_x))
# keypoints_y = list(map(float,keypoints_y))
# keypoints_x = list(map(round,keypoints_x))
# keypoints_y = list(map(round,keypoints_y))

for ind_ in range(len(inds_3d_point)):
    inds_used_one_image = []
    keypoints_x_used_one_image = []
    keypoints_y_used_one_image = []
    
#     for _ind in range(len(list(map(int,inds_3d_point[ind_])))):
    for _ind in range(len(inds_3d_point[ind_])):

#         if list(map(int,inds_3d_point[ind_]))[_ind] != -1:
        if inds_3d_point[ind_][_ind] != '-1':
#             inds_used_one_image.append(list(map(int,inds_3d_point[ind_]))[_ind])
#             keypoints_x_used_one_image.append(list(map(float,keypoints_x[ind_]))[_ind])
#             keypoints_y_used_one_image.append(list(map(float,keypoints_y[ind_]))[_ind])
            inds_used_one_image.append(inds_3d_point[ind_][_ind])
            keypoints_x_used_one_image.append(keypoints_x[ind_][_ind])
            keypoints_y_used_one_image.append(keypoints_y[ind_][_ind])
            
    keypoints_x_used.append(keypoints_x_used_one_image)
    keypoints_y_used.append(keypoints_y_used_one_image)
    inds_used.append(inds_used_one_image)

print(len(keypoints_x_used))
print(len(keypoints_y_used))
print(len(inds_used))

In [9]:
imid = image_names.index(imname)
Q_imid = Quats[imid]
t_imid = Ts[imid]
rot_mat_imid, trans_mat_imid, T_imid = quaternion_rotation_matrix(list(map(float,Q_imid)), list(map(float,t_imid)))
K_imid = getCameraIntrinsicMatrix(list(map(float,params[imid])))

P_imid = np.matmul(K_imid,T_imid[:3,:]) #getProjMat
P_imid = np.vstack((P_imid,[0, 0, 0, 1])) 
inv_P_temp_imid = np.linalg.pinv(P_imid)
inv_P_imid = inv_P_temp_imid[:3,:] #getInvProjMat

num_2d_imid = len(keypoints_x[imid][:])

row_arr = list(map(float,keypoints_x_used[imid][:]))
col_arr = list(map(float,keypoints_y_used[imid][:]))
# row_arr = keypoints_x_used[imid][:]
# col_arr = keypoints_y_used[imid][:]

points_calculated_3d = []
# for i in range(len(num_2d_imid)):
for i in range(len(10)):

#     row = round(row_arr[i])
#     col = round(col_arr[i])
#     depth = depthmap[row, col] #row, col are 2d coordinates of a point in the image
    row = row_arr[i]
    col = col_arr[i]
    depth = 1
    point_3d = np.matmul(inv_P_imid, [col * depth, row * depth, depth, 1].T)
    points_calculated_3d.append(point_3d)

IndexError: list index out of range

In [ ]:
len(np.array(keypoints_x_used)[3])
print(len(inds_used))

In [ ]:
print('3d reconstructed point coords corresponding to 2d key points in image 00030.jpg', points_calculated_3d)